# The Pol-InSAR course - April 15, 2024 
# Synthetic Aperture Radar (SAR) 
# Part 2: Azimuth Resolution

In [ ]:
# --- Download exercise data & import reader function

from pysarpro import io, data
from pysarpro.io import rrat

data.download_all(directory="/projects", pattern=r'^data/01-sar')

# ---

In [ ]:
# --- To get started, we set the ipympl backend, which makes matplotlib plots interactive.
# --- We do this using a magic command, starting with %.
%matplotlib widget

# --- import libraries
import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq, fftshift, ifft

# --- Useful functions

def rect(t, T0):
    # Implements y = rect(t/T0) = 1 if abs(t/T0) <= 0.5, 0 otherwise 
    # All input / outputs are Numpy arrays
    
    N = t.shape[0]
    y = np.zeros(N, 'float32') 
    y[np.abs(t/T0) <= 0.5] = 1
    
    return y

# --- Constants

# speed of light
c0 = 2.9972190e+08


## Exercise 1

A typical P-band acquisition of DLR's F-SAR system is characterized by the following parameters:

In [ ]:
# --- Input parameters 

# SAR wavelength meters
lambda0 = 0.68901587
# platform velocity, meters/second
vp    = 102.89772    
# range at closest point
R0    = 6445.2197      
# pixel spacing in meters
azsp  = 0.90879266    
# number of pixels 
Nsaz  = 9984

The (unfocused) azimuth signal in */projects/data/01-sar/signal1_ac.npy* refers to 1 point-like scatterer. 
Which is the correct value for the azimuth resolution between 0.5 m and 5 m ?

### Solution

In [ ]:
#
# ...

# --- chirp signal
s0_t = np.exp(1j * np.pi * ka * time_a**2) * rect(time_a, Tsa)




## Exercise 2

Now load the azimuth signal in */projects/data/01-sar/signal2_ac.npy*, and focus it with the azimuth chirp calculated with the resolution determined in Exercise 1. How many scatterer do you see, and where?

### Solution

In [ ]:
# ...


## Exercise 3

We are now ready now to focus in both range and azimuth the real F-SAR P-band data already used last week !

1. Open the raw data image in the file */projects/data/01-sar/raw-img.rat*, and visualize it (only the amplitude).
2. Compress it in range using the chirp parameters provided below using the FFT. What do you see?
3. After range focusing, compress the result in azimuth, again using the FFT. What happens if the azimuth chirp is adapted in near range? what happens if the chirp is adapted in far range? And what if it is adapted at every range?

Tip:
Use the provided rat class to open the data - Example: img = rrat(*filename.rat*)

In [ ]:

# --- Inputs

# SAR wavelength
lambda0 = 0.68901587

# --- parameters for range focusing

# range chirp duration, s
T   = 1.0016000e-05
# range resolution, m
deltar  = 2.99
# range sampling, m
rsp = 1.1988876
# number of samples 
Nsr = 3772

# --- parameters for azimuth focusing

# azimut resolution, m
deltasa = 2/1.33 # 1.5
# platform velocity, meters/second
vp      = 102.89772    
# pixel spacing in meters
azsp    = 0.90879266    
# number of pixels 
Nsaz    = 9984
# range delay, s <---- NEW !!!
rd  = 4.3008000e-05
# the range delay is the amount of time between the chirp transmission
# and the receipt of the first backscattered signal ==> converted in meters it provides the minimum range


### Solution

In [ ]:
img = rrat('/projects/data/01-sar/raw-img.rat')


In [ ]:
# --- generate range chirp
time_r = np.linspace(0, Nsr-1, Nsr) * rsp / (c0/2)
time_r = time_r - np.max(time_r) / 2
kr = c0 / 2 / deltar / T
sr_t = np.exp(1j * np.pi * kr * time_r**2) * rect(time_r, T)

# --- range focusing
hr_t = np.conj(sr_t[::-1])
img_r = np.zeros(img.shape, 'complex64')

for nn in range(Nsaz) :
    
    u_fft = fft(img[nn, :]) * fft(hr_t)
    img_r[nn, :] = fftshift( ifft(u_fft) )
    
# --- plot before range focusing
plt.figure(figsize = (5, 8))
plt.imshow(np.flipud(abs(img)), vmin = 0, vmax = 2*np.mean(abs(img)), cmap = 'gray')
plt.xlabel('Range (pixels)')
plt.ylabel('Azimuth (pixels)')
plt.title('Raw data')

# --- plot after range focusing
plt.figure(figsize = (5, 8))
plt.imshow(np.flipud(abs(img_r)), vmin = 0, vmax = 2*np.mean(abs(img_r)), cmap = 'gray')
plt.xlabel('Range (pixels)')
plt.ylabel('Azimuth (pixels)')
plt.title('After range focusing')



In [ ]:
# --- focus in azimuth adapting to one range 




In [ ]:
# --- range-adaptive azimuth focusing

